# Decision Trees

Continuing our discussion of DTs.

## DTs for Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Load the California Housing dataset
california = fetch_california_housing()
X = california.data
y = california.target

type(X), type(y)

`X` and `y` are both numpy arrays. Let's build a dataframe to more easily assess the data.

In [ ]:
df = pd.DataFrame(california.data, columns=california.feature_names)
df['Price'] = california.target
df.head()

In [ ]:
print(california.DESCR)

### Split, Fit, Evaluate

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create and train a decision tree regressor
tree_reg = DecisionTreeRegressor(max_depth=3, random_state=42)
tree_reg.fit(X_train, y_train)

# Make predictions
y_pred = tree_reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {np.sqrt(mse):.2f}")
print(f"R² Score: {r2:.2f}")

### Plot the Tree

In [ ]:
plt.figure(figsize=(15, 10))
plot_tree(tree_reg, filled=True, feature_names=california.feature_names, rounded=True)
plt.title('Regression Tree for California Housing Prices')
plt.tight_layout()
plt.show()

# Print information about the tree
print(f"Number of nodes: {tree_reg.tree_.node_count}")
print(f"Tree depth: {tree_reg.get_depth()}")
print(f"Number of leaves: {tree_reg.get_n_leaves()}")

### Feature Importance

In [ ]:
importance = tree_reg.feature_importances_
plt.figure(figsize=(12, 6))
plt.bar(california.feature_names, importance)
plt.xticks(rotation=45)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance for Housing Price Prediction')
plt.tight_layout()
plt.show()

most_important_feature = np.argmax(importance)
print(f"Most important feature: {california.feature_names[most_important_feature]}")

In [ ]:
importance

### Plot Predictions Based Only on Most Important Feature

In [ ]:
# Create a simplified model using only the most important feature
X_single = X_train[:, [most_important_feature]]
single_tree = DecisionTreeRegressor(max_depth=3)
single_tree.fit(X_single, y_train)

# Create a range of values for plotting
X_range = np.linspace(min(X[:, most_important_feature]), 
                       max(X[:, most_important_feature]), 
                       1000).reshape(-1, 1)
y_range = single_tree.predict(X_range)

# Plot with a proper step function
plt.figure(figsize=(10, 6))
plt.scatter(X_train[:, most_important_feature], y_train, alpha=0.3, label='Training data')
plt.scatter(X_test[:, most_important_feature], y_test, alpha=0.3, label='Test data')
plt.step(X_range.flatten(), y_range, 'r-', where='post', linewidth=2, label='Tree predictions')

plt.xlabel(f'Feature: {california.feature_names[most_important_feature]}')
plt.ylabel('House Price (100k$)')
plt.title('Regression Tree Predictions vs Most Important Feature')
plt.legend()
plt.tight_layout()
plt.show()

Observations:

1. **Clear Decision Thresholds**: The vertical lines in the red prediction line occur at specific MedInc values where the decision tree makes splits (approximately at 1.0, 1.8, 2.6, 3.4, 4.3, 5.4, 6.8, and so on).

2. **Piecewise Constant Predictions**: Between any two thresholds, the tree makes exactly the same prediction for all houses regardless of small variations in income within that range.

3. **Increasing Steps**: Each "step up" represents a new leaf node with a higher predicted house price. The decision tree has correctly captured the overall positive relationship between median income and house prices.

4. **Simplified Approximation**: This stepped pattern shows how regression trees approximate continuous relationships with a series of flat regions - making them less flexible than something like linear regression for truly linear relationships, but more flexible for capturing non-linear patterns.

5. **Limited Resolution**: With max_depth=3, the tree can only make a limited number of splits, which is why you see relatively few steps rather than a more granular approximation.


## Regularization

### Post-Pruning

In [ ]:
# Function to evaluate a model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    return {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2
    }

#### Fit Models

Implementation is non-trivial in SKL. Here is the key bit:

- generate a full tree
- use `cost_complexity_pruning_path` function to calculate the exact alpha values where the optimal tree structure changes. Each alpha corresponds to a specific pruning decision.
- fit and evaluate one DT for each alpha value of interest
- use the best resulting model

In [ ]:
# First, generate a sequence of alphas to test
# Create a fully grown tree to use for pruning paths
full_tree = DecisionTreeRegressor(random_state=42)
path = full_tree.cost_complexity_pruning_path(X_train, y_train)

# Extract the alphas
ccp_alphas = path.ccp_alphas
ccp_alphas = ccp_alphas[:-1]  # Remove the last alpha which gives a trivial tree

# Limit the number of alphas to a reasonable range
if len(ccp_alphas) > 10:
    # Take a subset of alphas for demonstration
    indices = np.linspace(0, len(ccp_alphas) - 1, 10, dtype=int)
    ccp_alphas = ccp_alphas[indices]

# Create and evaluate models with different ccp_alpha values
models = []
results = []

for alpha in ccp_alphas:
    model = DecisionTreeRegressor(ccp_alpha=alpha, random_state=42)
    model.fit(X_train, y_train)
    models.append(model)
    
    # Store model statistics
    result = evaluate_model(model, X_train, X_test, y_train, y_test)
    result['alpha'] = alpha
    result['n_nodes'] = model.tree_.node_count
    result['depth'] = model.get_depth()
    results.append(result)

#### Extract Performance Metrics

In [ ]:
# Convert results to arrays for easier plotting
alphas = [r['alpha'] for r in results]
train_rmse = [r['train_rmse'] for r in results]
test_rmse = [r['test_rmse'] for r in results]
n_nodes = [r['n_nodes'] for r in results]

# Find the best alpha based on test performance
best_alpha_idx = np.argmin(test_rmse)
best_alpha = alphas[best_alpha_idx]
print(f"Best alpha: {best_alpha:.6f}")
print(f"Number of nodes: {n_nodes[best_alpha_idx]}")
print(f"Train RMSE: {train_rmse[best_alpha_idx]:.4f}")
print(f"Test RMSE: {test_rmse[best_alpha_idx]:.4f}")

Compare with our simple tree, with max depth 3, 15 nodes, 8 leaves → Test RMSE of 0.80!

#### Plot Results

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot RMSE
color = 'tab:blue'
ax1.set_xlabel('Alpha')
ax1.set_ylabel('RMSE', color=color)
ax1.plot(alphas, train_rmse, 'o-', color=color, label='Train RMSE')
ax1.plot(alphas, test_rmse, 's-', color='tab:orange', label='Test RMSE')
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xscale('log')

# Create a second y-axis for number of nodes
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Number of nodes', color=color)
ax2.plot(alphas, n_nodes, '^-', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Add legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + ['Number of nodes'], loc='upper left')

plt.title('Effect of ccp_alpha on Tree Complexity and Performance')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#### Look at Best Model

In [ ]:
best_model = DecisionTreeRegressor(ccp_alpha=best_alpha, random_state=42)
best_model.fit(X_train, y_train)

# Compare with an unpruned tree
unpruned_model = DecisionTreeRegressor(random_state=42)
unpruned_model.fit(X_train, y_train)
unpruned_result = evaluate_model(unpruned_model, X_train, X_test, y_train, y_test)

print("\nComparison:")
print(f"Unpruned tree - Nodes: {unpruned_model.tree_.node_count}, Depth: {unpruned_model.get_depth()}")
print(f"Pruned tree - Nodes: {best_model.tree_.node_count}, Depth: {best_model.get_depth()}")
print(f"Unpruned tree - Train RMSE: {unpruned_result['train_rmse']:.4f}, Test RMSE: {unpruned_result['test_rmse']:.4f}")
print(f"Pruned tree - Train RMSE: {train_rmse[best_alpha_idx]:.4f}, Test RMSE: {test_rmse[best_alpha_idx]:.4f}")

#### Visually Compare

In [ ]:
# Plot predictions for both models using the most important feature
feature_importances = best_model.feature_importances_
most_important_feature = np.argmax(feature_importances)
print(f"\nMost important feature: {california.feature_names[most_important_feature]}")

# Plot predictions
plt.figure(figsize=(12, 6))
plt.scatter(X_test[:, most_important_feature], y_test, alpha=0.3, label='Test data')

# Create a range of values for the most important feature
X_range = np.linspace(min(X[:, most_important_feature]), 
                       max(X[:, most_important_feature]), 
                       1000).reshape(-1, 1)
X_dummy = np.tile(np.median(X_test, axis=0), (X_range.shape[0], 1))
X_dummy[:, most_important_feature] = X_range[:, 0]

# Generate predictions for both models
y_unpruned = unpruned_model.predict(X_dummy)
y_pruned = best_model.predict(X_dummy)

# Plot the predictions
plt.plot(X_range, y_unpruned, 'r-', alpha=0.7, label='Unpruned tree predictions')
plt.plot(X_range, y_pruned, 'g-', linewidth=2, label='Pruned tree predictions')

plt.xlabel(f'Feature: {california.feature_names[most_important_feature]}')
plt.ylabel('House Price (100k$)')
plt.title('Comparison of Pruned vs Unpruned Tree Predictions')
plt.legend()
plt.tight_layout()
plt.show()

This result shows a clear demonstration of cost-complexity pruning and its effect on decision tree performance.

Top Graph (Alpha vs. RMSE and Tree Size):

1. **Tree Complexity (Green Line)**: As alpha increases, the number of nodes dramatically decreases from over 25,000 to nearly 0, showing how pruning effectively reduces tree size.

2. **Training RMSE (Blue Line)**: Starting near zero with the unpruned tree (showing overfitting), it gradually increases as the tree is pruned, indicating reduced ability to fit the training data perfectly.

3. **Test RMSE (Orange Line)**: This is the critical curve. It shows a slight but meaningful improvement (dip) around alpha = 0.000043, before worsening at higher alpha values. This indicates the sweet spot where pruning improves generalization.

Numerical Results:

- The best alpha (0.000043) reduced the tree from 27,665 nodes to 3,647 nodes (about 87% reduction)
- Depth decreased from 34 to 23
- Training RMSE increased from 0.0000 (perfect fit) to 0.2309 (expected with pruning)
- **Test RMSE improved from 0.7266 to 0.6967** - this is the key result showing improved generalization

Bottom Graph (Predictions Comparison):

- **Unpruned Tree (Red Line)**: Shows highly erratic, jagged predictions that attempt to fit every data point
- **Pruned Tree (Green Line)**: Shows smoother, more stable predictions that better capture the general trend

This demonstrates the classic bias-variance tradeoff in machine learning. The unpruned tree has low bias (fits training data perfectly) but high variance (generalizes poorly). The pruned tree accepts some bias (doesn't fit training data perfectly) but achieves lower variance, resulting in better overall performance on new data.

The optimal pruning level maintains the important structural aspects of the tree while eliminating branches that mainly fit noise in the training data.

### Pre-Pruning

Find the set of hyperparameters that result in the best performing tree.

#### Fit Models using `GridSearchCV`

Define the hyperparameter space and perform exhaustive search.

In [ ]:
from sklearn.model_selection import GridSearchCV
import time

# Define the parameter grid to search
param_grid = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8]
}

# Create a base decision tree regressor
tree_reg = DecisionTreeRegressor(random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    tree_reg,
    param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    return_train_score=True,
    n_jobs=-1  # Use all available cores
)

# Perform the grid search
print("Starting grid search...")
start_time = time.time()
grid_search.fit(X_train, y_train)
end_time = time.time()
elapsed_time = end_time - start_time
print("Grid search completed!")
print(f"Elapsed time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

**Note:** use `n_jobs=-1` to make use of all cpu cores. Without this setting, the search took 38.76 seconds on my 5yr old intel macbook. With it, the time was reduced to 7.6 seconds.

#### Extract Performance Metrics

In [ ]:
# Get the best parameters and model
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert back to RMSE from negative RMSE
best_model = grid_search.best_estimator_

print(f"Best parameters: {best_params}")
print(f"Best CV RMSE: {best_score:.4f}")

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R²: {test_r2:.4f}")

# Compare with default model
default_model = DecisionTreeRegressor(random_state=42)
default_model.fit(X_train, y_train)
default_pred = default_model.predict(X_test)
default_rmse = np.sqrt(mean_squared_error(y_test, default_pred))
default_r2 = r2_score(y_test, default_pred)

print("\nComparison with default model:")
print(f"Default model - Test RMSE: {default_rmse:.4f}, R²: {default_r2:.4f}")
print(f"Tuned model - Test RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")
print(f"Improvement: {((default_rmse - test_rmse) / default_rmse * 100):.2f}%")

Reminder:
- CV RMSE is used to find the best parameter set, using CV on the training data
- Test RMSE is the estimated prediction error for the resulting model, based on the test data

The process is:

1. Split data into training and test sets
2. Use cross-validation within the training set to find the best hyperparameters (CV RMSE)
3. *Train the final model with the best hyperparameters on the entire training set*
4. Evaluate this model on the test set (Test RMSE)

#### Make Fancy Graphs

In [ ]:
# Plot results of grid search for max_depth
plt.figure(figsize=(12, 6))

# Extract results for different max_depth values (averaging over other parameters)
results = grid_search.cv_results_
max_depths = param_grid['max_depth']
mean_test_scores = {}
mean_train_scores = {}

for depth in max_depths:
    # Get indices where max_depth equals current depth
    indices = [i for i, params in enumerate(results['params']) 
               if params['max_depth'] == depth]
    
    # Calculate average scores for this depth
    mean_test_scores[depth] = -np.mean([results['mean_test_score'][i] for i in indices])
    mean_train_scores[depth] = -np.mean([results['mean_train_score'][i] for i in indices])

# Convert None to a plottable value (fixing the TypeError)
# First find the maximum numeric depth
numeric_depths = [d for d in max_depths if d is not None]
max_numeric_depth = max(numeric_depths) if numeric_depths else 0
# Then use this to replace None with a value that's slightly larger
plot_depths = [d if d is not None else max_numeric_depth + 5 for d in max_depths]

test_scores = [mean_test_scores[d] for d in max_depths]
train_scores = [mean_train_scores[d] for d in max_depths]

# Plot
plt.figure(figsize=(12, 6))
plt.plot(plot_depths, train_scores, 'o-', label='Training RMSE')
plt.plot(plot_depths, test_scores, 's-', label='Cross-validation RMSE')

plt.xlabel('max_depth')
plt.ylabel('RMSE')
plt.title('Effect of max_depth on Model Performance')
plt.legend()
plt.grid(True, alpha=0.3)

# Mark the best depth
best_depth = best_params['max_depth']
best_depth_value = best_depth if best_depth is not None else max_numeric_depth + 5
plt.axvline(x=best_depth_value, color='r', linestyle='--', alpha=0.5)
plt.text(best_depth_value, min(test_scores), f'Best: {best_depth}', 
         rotation=90, verticalalignment='bottom')

plt.tight_layout()
plt.show()

This graph shows how the max_depth parameter affects model performance for decision trees on the California Housing dataset:

1. **Training vs. Cross-validation Performance**: The blue line (Training RMSE) decreases continuously as max_depth increases, while the orange line (Cross-validation RMSE) initially decreases but then reaches a minimum and begins to increase slightly.

2. **Optimal Depth**: The vertical dashed line marks the best max_depth value of 15, which was identified by the grid search. This is where cross-validation RMSE is minimized.

3. **Overfitting Pattern**: This is a classic illustration of the bias-variance tradeoff:
   - At low depths (left side), both training and validation errors are high (underfitting)
   - As depth increases, both errors initially decrease
   - Beyond depth 10, training error continues to decrease substantially while validation error flattens and slightly increases (beginning of overfitting)
   - At depth 25, there's a large gap between training and validation performance (clear overfitting)

4. **Diminishing Returns**: The validation curve flattens considerably after depth 10, suggesting minimal improvement from adding additional complexity beyond this point.

The graph effectively supports the grid search results, showing that a max_depth of 15 provides good performance on unseen data while avoiding the severe overfitting that occurs with unlimited depth trees. This visualization helps explain why the tuned model achieved a 14.17% improvement over the default model.

But `max_depth` of 10 appears to be the best choice - minimal CV RMSE. What gives?

*The graph shows the average performance across all combinations of other parameters (min_samples_split and min_samples_leaf) for each max_depth value. However, the grid search selects the best specific combination of all parameters.*


#### `RandomSearchCV`

Grid search tests all possible combination of parameters. Can take a long time, but results in global optimum (within the specified space).

Random approaches can search a wider space in less time, but do not guarantee a global optimum.

Instead of arrays of parameter values, `RandomSearchCV` uses values drawn from distributions (can be continuous if appropriate).

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# specify parameters as distributions!!
param_dist = {
    'max_depth': randint(3, 30),  # Random integers between 3 and 30
    'min_samples_split': randint(2, 30),
    'min_samples_leaf': randint(1, 15)
}

# compare with prior grid
# 'max_depth': [3, 5, 7, 10, 15, 20, None],
# 'min_samples_split': [2, 5, 10, 20],
# 'min_samples_leaf': [1, 2, 4, 8]

# Create base decision tree regressor
tree_random = DecisionTreeRegressor(random_state=42)

# Set up RandomizedSearchCV
# Use 100 iterations to sample a good portion of the parameter space
random_search = RandomizedSearchCV(
    tree_random,
    param_dist,
    n_iter=100,
    cv=5,
    scoring='neg_root_mean_squared_error',
    return_train_score=True,
    random_state=42,
    n_jobs=-1
)

# Perform random search with timing
print("Starting random search...")
random_start_time = time.time()
random_search.fit(X_train, y_train)
random_end_time = time.time()
random_elapsed_time = random_end_time - random_start_time
print("Random search completed!")
print(f"Random search time: {random_elapsed_time:.2f} seconds ({random_elapsed_time/60:.2f} minutes)")

# Get best parameters and score
random_best_params = random_search.best_params_
random_best_score = -random_search.best_score_  # Convert back to RMSE from negative RMSE
random_best_model = random_search.best_estimator_

print("\nBest parameters from random search:")
print(f"Random search parameters: {random_best_params}")
print(f"Random search CV RMSE: {random_best_score:.4f}")

# Evaluate on test set
random_pred = random_best_model.predict(X_test)
random_test_rmse = np.sqrt(mean_squared_error(y_test, random_pred))
random_test_r2 = r2_score(y_test, random_pred)

print("\nTest performance:")
print(f"Random search - RMSE: {random_test_rmse:.4f}, R²: {random_test_r2:.4f}")

Here the time difference is not significant, but it can be a huge benefit.

Moreover, we tested a wider space in less time and got better results!

#### YOLO

Here I just let Claude rip and ran what it came up with. Kept it because the visualizations are interesting and the implications are powerful.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import time
from scipy.stats import randint, uniform

# Load the California Housing dataset
california = fetch_california_housing()
X = california.data
y = california.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the parameter grids
param_grid = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8]
}

# For random search, we use distributions rather than fixed values
param_dist = {
    'max_depth': randint(3, 30),  # Random integers between 3 and 30
    'min_samples_split': randint(2, 30),
    'min_samples_leaf': randint(1, 15)
}

# Create base decision tree regressors
tree_grid = DecisionTreeRegressor(random_state=42)
tree_random = DecisionTreeRegressor(random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    tree_grid,
    param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    return_train_score=True,
    n_jobs=-1
)

# Set up RandomizedSearchCV
# We'll use 100 iterations to sample a good portion of the parameter space
random_search = RandomizedSearchCV(
    tree_random,
    param_dist,
    n_iter=100,
    cv=5,
    scoring='neg_root_mean_squared_error',
    return_train_score=True,
    random_state=42,
    n_jobs=-1
)

# Perform grid search with timing
print("Starting grid search...")
grid_start_time = time.time()
grid_search.fit(X_train, y_train)
grid_end_time = time.time()
grid_elapsed_time = grid_end_time - grid_start_time
print("Grid search completed!")
print(f"Grid search time: {grid_elapsed_time:.2f} seconds ({grid_elapsed_time/60:.2f} minutes)")

# Perform random search with timing
print("\nStarting random search...")
random_start_time = time.time()
random_search.fit(X_train, y_train)
random_end_time = time.time()
random_elapsed_time = random_end_time - random_start_time
print("Random search completed!")
print(f"Random search time: {random_elapsed_time:.2f} seconds ({random_elapsed_time/60:.2f} minutes)")

# Compare best parameters and scores
grid_best_params = grid_search.best_params_
grid_best_score = -grid_search.best_score_  # Convert back to RMSE from negative RMSE
grid_best_model = grid_search.best_estimator_

random_best_params = random_search.best_params_
random_best_score = -random_search.best_score_  # Convert back to RMSE from negative RMSE
random_best_model = random_search.best_estimator_

print("\nBest parameters comparison:")
print(f"Grid search: {grid_best_params}")
print(f"Random search: {random_best_params}")

print("\nCV score comparison (RMSE):")
print(f"Grid search: {grid_best_score:.4f}")
print(f"Random search: {random_best_score:.4f}")

# Evaluate models on test set
grid_pred = grid_best_model.predict(X_test)
grid_test_rmse = np.sqrt(mean_squared_error(y_test, grid_pred))
grid_test_r2 = r2_score(y_test, grid_pred)

random_pred = random_best_model.predict(X_test)
random_test_rmse = np.sqrt(mean_squared_error(y_test, random_pred))
random_test_r2 = r2_score(y_test, random_pred)

print("\nTest performance comparison:")
print(f"Grid search - RMSE: {grid_test_rmse:.4f}, R²: {grid_test_r2:.4f}")
print(f"Random search - RMSE: {random_test_rmse:.4f}, R²: {random_test_r2:.4f}")

# Compare efficiency
total_grid_combinations = len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf'])
print("\nEfficiency comparison:")
print(f"Grid search: {total_grid_combinations} combinations evaluated in {grid_elapsed_time:.2f} seconds")
print(f"Random search: 100 combinations evaluated in {random_elapsed_time:.2f} seconds")
print(f"Time per combination - Grid: {grid_elapsed_time/total_grid_combinations:.2f}s, Random: {random_elapsed_time/100:.2f}s")
print(f"Speed improvement: {(grid_elapsed_time/random_elapsed_time):.2f}x")

# Create visualization to compare performance across iterations
plt.figure(figsize=(14, 8))

# Create a subplot for the learning curves
plt.subplot(2, 1, 1)
grid_results = pd.DataFrame(grid_search.cv_results_)
random_results = pd.DataFrame(random_search.cv_results_)

# Sort both results by mean test score
grid_sorted = grid_results.sort_values('mean_test_score', ascending=False)
random_sorted = random_results.sort_values('mean_test_score', ascending=False)

# Plot learning curves - top N results
top_n = min(50, len(grid_sorted), len(random_sorted))
x_grid = np.arange(top_n)
x_random = np.arange(top_n)

plt.plot(x_grid, -grid_sorted['mean_test_score'].values[:top_n], 'b-', label='Grid Search CV Score')
plt.plot(x_random, -random_sorted['mean_test_score'].values[:top_n], 'r-', label='Random Search CV Score')

plt.xlabel('Rank of Model Configuration')
plt.ylabel('RMSE (Cross-Validation)')
plt.title('Top Model Configurations Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Create a subplot to show parameter space coverage
plt.subplot(2, 1, 2)

# Extract unique parameter values from both searches
grid_max_depths = grid_results['param_max_depth'].unique()
random_max_depths = random_results['param_max_depth'].unique()
all_max_depths = sorted(set([d for d in grid_max_depths if d is not None] + 
                          [d for d in random_max_depths if d is not None]))

# Create scatter plot of evaluated configurations
plt.scatter(grid_results['param_max_depth'].fillna(0), 
           grid_results['param_min_samples_leaf'],
           c=-grid_results['mean_test_score'], marker='o', s=50, 
           cmap='Blues', alpha=0.7, label='Grid Search')

plt.scatter(random_results['param_max_depth'].fillna(0), 
           random_results['param_min_samples_leaf'],
           c=-random_results['mean_test_score'], marker='x', s=50, 
           cmap='Reds', alpha=0.7, label='Random Search')

plt.colorbar(label='RMSE')
plt.xlabel('max_depth (0 = None)')
plt.ylabel('min_samples_leaf')
plt.title('Parameter Space Coverage')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Top Graph: Model Configurations Comparison

1. **Overall Performance**: The random search (red line) consistently achieved better (lower) RMSE scores than the grid search (blue line) across all configurations.

2. **Best Models**: The top-ranked random search model achieved an RMSE around 0.625, while the best grid search model's RMSE was about 0.63 - making random search slightly better.

3. **Stability**: The random search line shows a more gradual increase in RMSE as we move to lower-ranked configurations, suggesting more consistent performance across the parameter space.

4. **Configuration Range**: The grid search line shows more variability and steeper increases, indicating greater performance differences between the best and worst configurations.

Bottom Graph: Parameter Space Coverage

1. **Exploration Pattern**: 
   - Grid search (blue dots) appears at specific, regular intervals, showing the structured nature of grid search
   - Random search (red X's) is scattered throughout the parameter space, showing its wider exploration

2. **Density**: Random search has explored areas of the parameter space that grid search missed completely, particularly combinations with higher min_samples_leaf values (8-12 range).

3. **Best Configurations**: The darker colored points (indicating lower RMSE) for random search appear in several regions, including some parameter combinations that grid search didn't evaluate.

4. **Parameter Importance**: Both approaches found good models in the max_depth range of 3-10, but random search discovered that higher min_samples_leaf values (around 10-12) could also produce good results.

Key Takeaways:

1. Random search found better overall solutions while testing fewer combinations
2. Random search explored a broader and more diverse parameter space
3. Grid search was limited to its predefined grid points, missing potentially valuable regions
4. Random search likely benefited from examining parameter combinations outside the grid search's discrete values

This perfectly illustrates why random search can be more efficient - it found better models by exploring the parameter space more effectively, despite evaluating fewer total combinations.